# Lineare Regressionsmodelle
In dieser Übung versuchen wir den Verkaufspreis von Autos mittles verschiedener Regressionsmodelle vorherzusagen. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
df = pd.read_csv("/kaggle/input/car-price-prediction/CarPrice_Assignment.csv", index_col="car_ID")

## 2. Vorbereitung der Daten
Im ersten Teil beschäftigen wir uns mit der Vorbereitung der Daten. Das Datenset hat viele Features, in dieser Übung benutzen wir allerdings nur ein paar davon.

Zuerst erstellen wir einen neuen `DataFrame` mit den Features `curbweight, enginesize, highwaympg, horsepower, citympg, peakrpm` und `price`.

In [ ]:
data = df[['curbweight', 'enginesize', 'highwaympg', 'horsepower', 'citympg', 'peakrpm', 'price']]

- Benutze `pandas.plotting.scatter_matrix` um einen tieferen Einblick in die Daten zu bekommen. Gibt es Features, welche nützlicher sind als andere?
- Berechne eine Korrelationsmatrix. Benutze dafür `DataFrame.corr()`.

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
pd.plotting.scatter_matrix(data)
data.corr()
df.corr()
df.info
data.info
data.info(verbose = True, show_counts = True)

Zuletzt wollen wir unser Datenset in ein *Feature*-Datenset und ein *Target* Datenset teilen, um anschließend ein Trainings- und ein Testset zu erstellen. Diese können wir diese beiden Sets verwenden um einige Regressionsmodelle zu testen: 

In [ ]:
from sklearn.model_selection import train_test_split
X = data[['curbweight', 'enginesize', 'highwaympg', 'horsepower', 'citympg', 'peakrpm']]
y = data.price
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

## 2a. Lineare Regression
- Benutze `X_train, y_train` und `sklearn.linear_model.LinearRegression` um eine lineare Regression zu trainieren.
- Validiere das Modell mit `sklearn.model_selection.cross_val_score`. Solltest du das Trainingsset oder das Testset verwenden?
- Benutze `numpy.mean` um den durchschnittlichen Score zu berechnen.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np

#intercept_ is the expected mean value of y when all X are 0
#coef_ is used to estimate the coefficients of a linear regression problem
#cross_val_score is messuring the accuracy
#np.mean compute the arithmetric mean
# we should use the train set only, or else the model will be biased

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_predt = lin_reg.predict(X_train)
print(y_predt)
print(lin_reg.intercept_)
print(lin_reg.coef_)
print("Model validation",cross_val_score(lin_reg, X_train, y_train))
print("Durchschnittlicher Score ",np.mean(cross_val_score(lin_reg, X_train, y_train)))
compare = pd.DataFrame({'Truevaly': y_train, 'Predictedvaly' : y_predt})
print(compare)



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy

## 2b. Polynomregression
Kreiere eine `Pipeline` und benutze `PolynomialFeatures` um ein Polynomregressions-Modell zu erstellen.
Berechne den durchschnittlichen Cross Validation Score.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import numpy
std_scaler = StandardScaler()
polybig_features=PolynomialFeatures(degree = 2, include_bias=False).fit(X_train,y_train)
poly_reg = Pipeline([('poly_features',polybig_features),('std_scaler', std_scaler),('lin_reg',lin_reg)])
poly_reg.fit(X_train,y_train)
print(cross_val_score(poly_reg, X_train, y_train))
print("Durchschnittlicher cross_val_score",np.mean(cross_val_score(poly_reg, X_train, y_train)))

Wir können eine Cross Validation mittles `GridSearchCV` machen. Um die inneren Parameter der `Pipeline` zu variieren können wir einen doppelten Unterstrich verwenden: `poly_features__degree`.

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(poly_reg, {'poly_features__degree': [1, 2, 3]})
grid_search.fit(X_train, y_train)
print(cross_val_score(grid_search,X_train,y_train))
print(f'best parameter {grid_search.best_params_}')

In [ ]:
polyvar_features = PolynomialFeatures

## 2c. Ridge Regression
- Erstelle eine `Pipeline` und benutze `PolynomialFeatures` um ein Polynomregressions-Modell zu erstellen. Benutze diesmal eine `Ridge` Regression. 
- Benutze `GridSearchCV` um den Hyperparameter $\alpha$ zu bestimmen.
- Berechne den durchschnittlichen Cross Validation Score mit dem besten $\alpha$-Wert. 
- Regularisierte Modelle funktionieren oft besser, wenn man die Daten zusätzlich skaliert. Benutze `StandardScaler` und vergleiche den Cross Validation Score. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge


polybig_features = PolynomialFeatures(degree=2, include_bias=False).fit(X_train,y_train)
std_scaler = StandardScaler()
ridge_reg = Pipeline([('poly_features',polybig_features),('regul_reg',Ridge())])
ridge_reg.fit(X_test,y_test)
grid_search = GridSearchCV(ridge_reg,{'regul_reg__alpha': [x for x in range (0,100)]})
grid_search.fit(X_train,y_train)
print(f'best parameter {grid_search.best_params_}')
print(cross_val_score(ridge_reg,X_train,y_train))
print(np.mean(cross_val_score(ridge_reg, X_train, y_train)))

ridge_regscale = Pipeline([('poly_features',polybig_features),('std_scaler', std_scaler),('regul_regscaler',Ridge())])
ridge_regscale.fit(X_test,y_test)
grid_searchscaler = GridSearchCV(ridge_regscale,{'regul_regscaler__alpha': [x for x in range (0,100)]})
grid_searchscaler.fit(X_train,y_train)
print(f'best parameter {grid_searchscaler.best_params_}')
print(cross_val_score(ridge_regscale,X_train,y_train))

ridge_regwcalcalpha = Pipeline([('poly_features',polybig_features),('regul_regscaler',Ridge(alpha=42))])
ridge_regwcalcalpha.fit(X_train,y_train)
print(np.mean(cross_val_score(ridge_regwcalcalpha, X_train, y_train)))
ridge_regscalewcalcalpha = Pipeline([('poly_features',polybig_features),('std_scaler', std_scaler),('regul_regscaler',Ridge(alpha=1))])
ridge_regscalewcalcalpha.fit(X_train,y_train)
print(np.mean(cross_val_score(ridge_regscalewcalcalpha, X_train, y_train)))

compare = pd.DataFrame({np.mean(cross_val_score(ridge_regwcalcalpha, X_train, y_train)),np.mean(cross_val_score(ridge_regscalewcalcalpha, X_train, y_train))})
print(compare)

#LinAlgWarning occures because the Standardscaler was not included in the ridge_reg Pipeline on purpose for comparison

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

## 2d. Elastic Net
Benutze `PolynomialFeatures` mit einem höheren Grad ($\gt 2$) und erstelle ein Elastic Net Modell. 
- Benutze dafür `SGDRegressor` mit `penalty='elasticnet'`. Benutze weiters `GridSearchCV` um die Hyperparameter `alpha` und `l1_ratio` zu wählen. 
- Was sind die besten Hyperparameter. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures

polybig_features = PolynomialFeatures(degree=4, include_bias=False).fit(X_test,y_test)
std_scaler = StandardScaler()
elastic_net = Pipeline([
    ("poly_features", polybig_features),
    ("std_scaler", std_scaler),
    ("elastic_net", SGDRegressor(penalty='elasticnet'))])

elastic_net.fit(X_train, y_train)
grid_search = GridSearchCV(elastic_net,{'elastic_net__alpha': [x/100 for x in range (0,100)],'elastic_net__l1_ratio': [0,0.05,0.1,0.2,0.5,0.8,0.9,1]}, cv=10)
grid_search.fit(X_train,y_train)
print(f'best parameter {grid_search.best_params_}')

cv_results = cross_validate(elastic_net, X, y, cv = 3)
sorted(cv_results.keys())
print(cross_val_score(elastic_net, X_train, y_train).mean())


In [ ]:
grid_search = GridSearchCV(elastic_net, {'elastic_net__alpha': [x for x in range (0,100)]}, cv=10)
grid_search.fit(X_train, y_train)

Wir können die Ergebnisse der Cross Validation in einem `DataFrame` zusammenfassen:

In [ ]:
outcome = pd.DataFrame(grid_search.cv_results_).sort_values("rank_test_score").head()
print(outcome)

## 2e. Early Stopping
Erstelle ein regularisiertes Modell, benutze aber diesmal [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html?highlight=sgdregressor#sklearn-linear-model-sgdregressor) mit der Einstellung `early_stopping=True` für die Implementierung. 
- Benutze `GridSearchCV` um die Lernrate `eta0` zu bestimmen. 
- Erstelle wie zuvor einen `DataFrame` der Ergebnisse. 
- Im Falle eines schlechten *Scores*, versuche den Parameter `n_iter_no_change` zu erhöhen. Was macht dieser Parameter?

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.exceptions import ConvergenceWarning

sgd_reg = Pipeline([
    ("poly_features", PolynomialFeatures(degree=4)),
    ("std_scaler", StandardScaler()),
    ("sgd_reg", SGDRegressor(early_stopping = True))
])

sgd_reg.fit(X_train, y_train)
grid_searcheta0 = GridSearchCV(sgd_reg,{'sgd_reg__eta0': [x for x in range (0,100)],'sgd_reg__n_iter_no_change' : [x for x in range (1,10)]}, cv=10)
sgd_reg.get_params().keys()
cv_results = cross_validate(sgd_reg, X, y, cv = 3)
sorted(cv_results.keys())
result = pd.DataFrame(grid_search.cv_results_).sort_values("rank_test_score").head()
print(result)
#wowo = pd.DataFrame(grid_searcheta0.cv_results_).sort_values("rank_test_score").head()
#print(wowo)

## 2f*. Feature Engineering (Zusatzaufgabe)
Wir wollen ein Feature `CarMake` hinzufügen. Der ursprüngliche Datensatz hat ein Feature `CarName`. Aus diesem Feature wollen wir die Automarke extrahieren. Allerdings sind die Daten fehlerhaft und können Rechtschreibfehler beinhalten.
- Wie viele Automarken gibt es im Datenset?

In [ ]:
from collections import Counter
make_mapping = {
    "maxda": "mazda",
    "Nissan": "nissan",
    "porcshce": "porsche",
    "toyouta": "toyota",
    "vokswagen": "vw",
    "volkswagen": "vw",
}

def get_make(car_name: str):
    make = car_name.split(' ')[0]
    return make_mapping.get(make, make)

features = ['curbweight', 'enginesize', 'highwaympg', 'horsepower', 'citympg', 'peakrpm']
X = data.loc[:, features]  # Erstellt eine Kopie
y = data.price.copy()

X["CarMake"] = df.CarName.map(get_make)
print(X["CarMake"].value_counts(dropna=False))
Counter(X["CarMake"])
print("Es gibt im DataSet 22 verschiedene Automarken")




- Benutze `sklearn.compose.ColumnTransformer(..., remainder='passthrough'`) und `sklearn.preprocessing.OneHotEncoder` um das kategorische Feature `CarName` in mehrere numerische Features zu verwandeln.
- Benutze `PolynomialFeatures` und anschließend `StandardScaler` und trainiere ein lineares Regressionsmodell deiner Wahl.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV

dunno_reg = Pipeline([
    ("poly_features", PolynomialFeatures(degree = 4)),
    ("std_scaler", StandardScaler),
    ("onehotencoder", OneHotEncoder)
    
])